In [45]:
import numpy as np
import pandas as pd
from sklearn.decomposition import PCA 
import os
import sys
sys.path.insert(0, os.path.abspath('../Libraries'))
import countries 



Getting Europe subregions...
Getting ISO data...
Countries data loaded.


In [694]:
#gather data sets

education = pd.read_csv("../Datasets/cleaned_education.csv")
technology = pd.read_csv("../Datasets/cleaned_technology.csv")
esi = pd.read_csv("../Datasets/ESI_Clean.csv", index_col=0)
tax_rate = pd.read_csv("../Datasets/cleaned_unemploymentTaxRateDataset.csv")
hours_worked = pd.read_csv("../Datasets/hours-actually-worked.csv")

In [695]:
# format hours worked to my purposes
hours_worked.drop(columns=["Country", "EMPSTAT", "Employment status","FREQUENCY", "Frequency","Time", "Unit Code", "Unit", "PowerCode Code", "PowerCode","Reference Period Code", "Reference Period", "Flag Codes", "Flags"], inplace=True)
hours_worked = pd.pivot_table(hours_worked, index="COUNTRY", columns="TIME", values="Value")
hours_worked.head()

TIME,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021
COUNTRY,,,,,,,,,,,,
AUS,1755.947546,1753.054164,1752.817330,1746.821146,1745.930470,1732.539866,1721.170727,1722.766006,1719.374012,1709.212114,1676.783056,1686.793927
AUT,1510.000000,1512.500000,1494.000000,1481.500000,1474.000000,1457.500000,1469.000000,1459.500000,1461.500000,1469.452826,1361.531738,1406.803068
BEL,1503.000000,1516.500000,1513.500000,1511.000000,1508.000000,1503.000000,1504.500000,1509.500000,1512.500000,1510.000000,1398.000000,1446.563193
CAN,1717.572240,1716.529704,1725.404215,1719.184365,1714.883890,1716.056196,1711.480750,1700.982313,1715.391403,1697.307754,1658.463386,1695.293631
CHE,1599.426959,1598.950747,1584.320240,1565.211725,1559.990134,1573.131432,1573.803493,1557.556987,1548.017728,1545.829958,1494.053219,1528.754309


In [696]:
tax_rate.head()

,COUNTRY,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021
0,AT,83.0,82.9,82.4,81.7,79.9,80.4,74.8,74.8,74.6,74.3,73.9,72.0,72.1,71.7,71.2,70.5,70.1
1,BE,85.2,84.9,85.3,85.5,92.1,91.0,90.7,91.9,98.1,97.6,96.9,95.8,95.8,93.8,93.0,92.4,91.6
2,CZ,65.7,63.5,59.9,59.6,71.3,71.4,71.7,71.6,71.6,71.7,71.8,72.0,72.3,72.5,72.7,72.8,70.8
3,DE,75.0,75.6,74.8,74.5,75.2,73.5,73.3,73.3,73.0,73.1,73.1,73.2,73.3,73.2,73.2,73.0,72.8
4,DK,91.2,90.6,89.5,88.6,88.8,89.2,88.7,89.4,89.4,89.6,89.6,89.8,89.9,89.3,88.4,87.9,87.2


for this task I will be using ISO-3 country codes


In [697]:
# use webscraper library to convert to ISO3
tax_rate.replace(to_replace=countries.get_alpha_2_list(), value = countries.get_alpha_3_list(), inplace=True)
# other data only goes from 2005-2018
tax_rate.drop(columns=["2019","2020","2021"], inplace=True)
tax_rate.set_index("COUNTRY", inplace=True)
education.drop(columns=["2019","2020","2021"], inplace=True)
education.set_index("geo", inplace=True)
technology.drop(columns=["2000","2001","2002","2003","2004"], inplace=True)


In [698]:
technology.set_index("Country_Code", inplace=True)
technology.head()

,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018
Country_Code,,,,,,,,,,,,,,
AUS,0.577590,0.558321,0.557928,0.544748,0.546733,0.561768,0.575363,0.570554,0.572189,0.555543,0.558735,0.546425,0.536043,0.535987
AUT,0.506048,0.492319,0.501761,0.498783,0.497396,0.525531,0.539327,0.547373,0.540696,0.522309,0.530957,0.497000,0.498190,0.495123
BEL,0.474615,0.453861,0.459687,0.450338,0.456328,0.477225,0.486988,0.486720,0.483297,0.478380,0.480012,0.462487,0.454118,0.453876
BGR,0.375573,0.386792,0.411913,0.413827,0.416350,0.427244,0.432422,0.442906,0.441728,0.424637,0.426063,0.414307,0.418258,0.419256
CAN,0.547830,0.527691,0.522781,0.518057,0.521169,0.520733,0.525753,0.524708,0.519229,0.504916,0.508174,0.493125,0.494192,0.494861


In [699]:


esi.columns=esi.iloc[0]

esi.drop(index="Month-Year", axis=0, inplace=True)
esi.head()


Month-Year,Jan-05,Feb-05,Mar-05,Apr-05,May-05,Jun-05,Jul-05,Aug-05,Sep-05,Oct-05,...,May-22,Jun-22,Jul-22,Aug-22,Sep-22,Oct-22,Nov-22,Dec-22,Jan-23,Feb-23
EU.ESI,100.4,99.4,98.4,98.1,97.3,97.4,98.5,99.8,100.3,102.0,...,103.6,101.9,97.8,97.1,93.9,92.8,94.1,95.6,97.8,97.8
EA.ESI,100.1,98.9,97.9,97.8,96.7,96.9,98.2,99.2,99.9,101.9,...,104.2,103.0,98.6,97.8,94.4,93.8,95.1,97.1,99.8,99.7
BE.ESI,105.3,104.4,102.6,98.1,94.6,97.0,94.8,96.9,99.9,104.2,...,97.8,97.7,98.0,92.9,86.0,83.4,82.4,88.0,87.4,88.9
BG.ESI,112.4,106.2,106.3,108.1,108.7,112.4,109.9,101.7,107.3,103.4,...,95.0,98.7,100.0,99.4,99.4,100.1,103.4,103.5,101.6,102.1
CZ.ESI,108.9,108.6,105.4,105.2,108.0,105.9,110.2,106.9,109.1,110.0,...,97.0,95.0,89.5,87.7,89.5,83.9,82.2,83.3,88.5,92.0


In [700]:
# finally, ESI has data for every month, so we pick one value per year to align
# hmm... I think I will choose ... June!

esi = esi.filter(regex="Jun")

esi.columns=list(range(2005,2023))
esi.head()

,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022
EU.ESI,97.4,108.3,111.0,94.9,77.1,97.2,100.9,89.9,90.0,98.8,102.4,103.6,110.1,111.0,102.8,74.1,114.1,101.9
EA.ESI,96.9,108.0,110.6,94.6,76.8,96.5,101.3,89.8,89.6,98.5,102.4,103.4,109.9,110.8,102.5,75.2,115.2,103.0
BE.ESI,97.0,110.5,116.5,100.9,81.3,99.3,104.7,89.5,89.7,100.7,101.1,108.3,106.8,106.3,96.5,70.0,117.6,97.7
BG.ESI,112.4,111.2,120.4,118.4,87.2,86.4,91.9,93.8,90.0,96.7,100.6,102.3,104.8,106.1,99.4,74.2,99.5,98.7
CZ.ESI,105.9,112.9,114.6,104.0,81.0,96.6,94.5,87.9,87.0,99.3,104.6,103.7,105.9,108.1,102.1,66.6,104.8,95.0


In [701]:
esi.index = esi.index.str.replace(r".ESI", "")
esi.drop(labels=["EU","EA"], axis=0, inplace=True)
esi.reset_index
esi.head()


C:\Users\harve\AppData\Local\Temp\ipykernel_6796\102450991.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  esi.index = esi.index.str.replace(r".ESI", "")


,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022
BE,97.0,110.5,116.5,100.9,81.3,99.3,104.7,89.5,89.7,100.7,101.1,108.3,106.8,106.3,96.5,70.0,117.6,97.7
BG,112.4,111.2,120.4,118.4,87.2,86.4,91.9,93.8,90.0,96.7,100.6,102.3,104.8,106.1,99.4,74.2,99.5,98.7
CZ,105.9,112.9,114.6,104.0,81.0,96.6,94.5,87.9,87.0,99.3,104.6,103.7,105.9,108.1,102.1,66.6,104.8,95.0
DK,100.2,114.9,100.7,87.4,85.8,109.6,104.4,90.6,98.4,97.8,102.1,105.7,111.2,111.4,102.8,76.5,114.1,88.5
DE,91.4,104.2,108.0,97.7,79.3,103.1,110.2,101.0,100.3,104.5,105.9,106.0,111.5,111.2,102.0,80.9,114.9,104.6


In [702]:

for code in esi.index:
    if code not in countries.get_alpha_2_list():
        esi.drop(labels=code, axis = 0, inplace=True)

esi.rename(countries.convert_alpha_2_to_3, axis="index", inplace=True)
esi.head()


,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022
BEL,97.0,110.5,116.5,100.9,81.3,99.3,104.7,89.5,89.7,100.7,101.1,108.3,106.8,106.3,96.5,70.0,117.6,97.7
BGR,112.4,111.2,120.4,118.4,87.2,86.4,91.9,93.8,90.0,96.7,100.6,102.3,104.8,106.1,99.4,74.2,99.5,98.7
CZE,105.9,112.9,114.6,104.0,81.0,96.6,94.5,87.9,87.0,99.3,104.6,103.7,105.9,108.1,102.1,66.6,104.8,95.0
DNK,100.2,114.9,100.7,87.4,85.8,109.6,104.4,90.6,98.4,97.8,102.1,105.7,111.2,111.4,102.8,76.5,114.1,88.5
DEU,91.4,104.2,108.0,97.7,79.3,103.1,110.2,101.0,100.3,104.5,105.9,106.0,111.5,111.2,102.0,80.9,114.9,104.6


In [703]:

esi.drop(columns=["2019","2020","2021","2022"], inplace=True)
esi.head()

KeyError: "['2019', '2020', '2021', '2022'] not found in axis"

In [ ]:
# now I need to make a large table, doubly indexed by country and year

unique_countries = list(set(esi.index).intersection(set(education.index), set(technology.index), set(hours_worked.index), set(tax_rate.index)))

years = list((range(2005,2019)))


#axes = [np.array(unique_countries), np.array(years)]
#tuples = list(zip(*axes))
#m_idx = pd.MultiIndex.from_tuples(tuples, names=["Country","Year"])
#pca_df = pd.DataFrame(np.zeros(shape=(len(unique_countries)*len(years),5)), index=axes)
pca_df = pd.concat({"education":education, "technology":technology, "ESI":esi, "hours worked": hours_worked, "tax rate":tax_rate}, axis=1)
stacked_pca = pca_df.stack()
stacked_pca.dropna(axis=0)
stacked_pca
#pca_df.pivot_table()

TypeError: DataFrame.stack() got an unexpected keyword argument 'inplace'